# Homework 3 - Master's Degrees from all over!

**Knowledge!**

Nowadays, society expects everyone, especially you, to have a decent level of knowledge from a relatively wide range of topics. To do so, as you are all doing here, most students choose, after finishing their undergraduate studies, to keep on studying. The natural next step is the Master's Degree!
You will be working with a [webpage](https://www.findamasters.com) that provides information and evalutation on many of these master's degrees in many universities worldwide.  



# VERY VERY IMPORTANT
1. **!!! Read the entire homework before coding anything!!!**
2. *My solution is not better than yours, and yours is not better than mine*. In any data analysis task, there **is no** unique way to answer. For this reason, it is crucial (**necessary and mandatory**) that you describe any single decision you take and all your steps.
3. Once solving an exercise, comments about the obtained results are **mandatory**. We are not always explicit about where to focus your comments, but we will always want brief sentences about your discoveries.
4. We encourage using chatGPT (Bard, Bing, or any other Large Language Models (LLM) chatbot tool) as allies to help you solve your homework, and we were hoping you could learn how to use them properly. However, **using such tools when not explicitly allowed will be considered plagiarism and strictly prohibited**. 

Now that it is all well settled, let's get on with it!

## 1. Data collection

For this homework, there is no provided dataset. Instead, you have to build your own. Your search engine will run on text documents. So, here
we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a ```crawler.py``` module, a ```parser.py``` module, and a ```engine.py``` module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues! 

In [ ]:
import requests
from bs4 import BeautifulSoup
import os # To create folder
import time
from tqdm.notebook import tqdm
import pandas as pd
import crawler



### 1.1. Get the list of master's degree courses

We start with the list of courses to include in your corpus of documents. In particular, we focus on web scrapping the [MSc Degrees](https://www.findamasters.com/masters-degrees/msc-degrees/). Next, we want you to **collect the URL** associated with each site in the list from the previously collected list.
The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in **the first 400 pages** (each page has 15 courses, so you will end up with 6000 unique master's degree URLs).

The output of this step is a `.txt` file whose single line corresponds to the master's URL.

In [ ]:
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'
page_urls = [] # list containing all the master's degree URLs

# Get the master's degree URL for all 400 pages
for page_number in tqdm(range(1, 401), desc="Extracting URLs", unit="Page"):
    
   # URL of the page 
    page_url = f"{url}?PG={page_number}"

    try:
        # append to the list the 15 courses of each page
        page_urls.extend(crawler.extract_MSc(page_url))


    except requests.exceptions.RequestException as e:
        print(f"Error in web scraping: {e}")


In [ ]:
# Here we save the .txt file
with open('master_degree_urls.txt','w') as file:
    for url in page_urls:
        file.write(url + '\n')

### 1.2. Crawl master's degree pages

Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its `HTML` in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the downloaded `HTML` pages into folders. Each folder will contain the `HTML` of the courses on page 1, page 2, ... of the list of master's programs.
   
__Tip__: Due to the large number of pages you should download, you can use some methods that can help you shorten the time. If you employed a particular process or approach, kindly describe it.
 

In [ ]:
# Here we create 400 folders on our computer and save 15 html files in each one of them

# Open the txt file and retrieve each URL
with open('/Users/Francesco/Desktop/ADM/HW3/master_degree_urls.txt', 'r') as file:
    urls = [line.strip() for line in file]


# Initialize variables to save 15 HTML files in each folder
i = 0
number = 1

for url in tqdm(urls, desc="Downloading HTML Files", unit="URL"):
    
    # To each URL, we need to add the base root of the website
    url = 'https://www.findamasters.com' + url

    # if I have saved 15 files, change folder
    if i == 15:
        i = 0
        number += 1

    folder_path = f'/Users/Francesco/Desktop/ADM/HW3/Page{number}'

    # Ensure the folder exists, create it if it doesn't
    os.makedirs(folder_path, exist_ok=True)

    # Get the content of the website
    r = requests.get(url)
    content = r.content

    time.sleep(1)  # Add a delay of 1 second between requests to avoid getting banned

    # Create a suitable name
    url = f'course_{i}'

    # Combine the folder_path and file_name to get the full path where to save the file
    file_path = os.path.join(folder_path, url)

    with open(f'{file_path}.html', 'wb') as html_file:
        html_file.write(content)

    i += 1

    time.sleep(1) # Add a delay of 1 second between requests to avoid getting banned


### 1.3 Parse downloaded pages

At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format is as follows:

1. Course Name (to save as ```courseName```): string;
2. University (to save as ```universityName```): string;
3. Faculty (to save as ```facultyName```): string
4. Full or Part Time (to save as ```isItFullTime```): string;
5. Short Description (to save as ```description```): string;
6. Start Date (to save as ```startDate```): string;
7. Fees (to save as ```fees```): string;
8. Modality (to save as ```modality```):string;
9. Duration (to save as ```duration```):string;
10. City (to save as ```city```): string;
11. Country (to save as ```country```): string;
12. Presence or online modality (to save as ```administration```): string;
13. Link to the page (to save as ```url```): string.
    
<p align="center">
<img src="img/example.jpeg" width = 1000>
</p>
This are the first rows of the scraped dataset:

<div style="overflow-x:auto;">
<table>
<thead>
  <tr>
    <th>index</th>
    <th>courseName</th>
    <th>universityName</th>
    <th>facultyName</th>
    <th>isItFullTime</th>
    <th>description</th>
    <th>startDate</th>
    <th>fees</th>
    <th>modality</th>
    <th>duration</th>
    <th>city</th>
    <th>country</th>
    <th>administration</th>
    <th>url</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>0</td>
    <td> Accounting and Finance - MSc</td>
    <td>University of Leeds</td>
    <td>Leeds University Business School</td>
    <td>Full time</td>
    <td>Businesses and governments rely on [...].</td>
    <td>September</td>
    <td>UK: £18,000 (Total) International: £34,750 (Total)</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Leeds</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-and-finance-msc/?i321d3232c3891">Link</a></td>
  </tr>
  <tr>
    <td>1</td>
    <td> Accounting, Accountability & Financial Management MSc</td>
    <td>King’s College London</td>
    <td>King’s Business School</td>
    <td>Full time</td>
    <td>Our Accounting, Accountability & Financial Management MSc course will provide [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>1 year FT</td>
    <td>London</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-accountability-and-financial-management-msc/?i132d7816c25522">Link</a></td>
  </tr>
  <tr>
    <td>2</td>
    <td> Accounting, Financial Management and Digital Business - MSc</td>
    <td>University of Reading</td>
    <td>Henley Business School</td>
    <td>Full time</td>
    <td>Embark on a professional accounting career [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Reading</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/accounting-financial-management-and-digital-business-msc/?i345d4286c351">Link</a></td>
  </tr>
  <tr>
    <td>3</td>
    <td> Addictions MSc</td>
    <td>King’s College London</td>
    <td>Institute of Psychiatry, Psychology and Neuroscience</td>
    <td>Full time</td>
    <td>Join us for an online session for prospective [...].</td>
    <td>September</td>
    <td>Please see the university website for further information on fees for this course.</td>
    <td>MSc</td>
    <td>One year FT</td>
    <td>London</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
    <td><a href="https://www.findamasters.com/masters-degrees/course/addictions-msc/?i132d4318c27100">Link</a></td>
  </tr>
  <tr>
    <td>4</td>
    <td> Advanced Chemical Engineering - MSc</td>
    <td>University of Leeds</td>
    <td>School of Chemical and Process Engineering</td>
    <td>Full time</td>
    <td>The Advanced Chemical Engineering MSc at Leeds [...].</td>
    <td>September</td>
    <td>UK: £13,750 (Total) International: £31,000 (Total)</td>
    <td>MSc</td>
    <td>1 year full time</td>
    <td>Leeds</td>
    <td>United Kingdom</td>
    <td>On Campus</td>
  </tr>
  <!-- Add more rows here as needed -->
</tbody>
</table>
</div>


For each master's degree, you create a `course_i.tsv` file of this structure:

```
courseName \t universityName \t  ... \t url
```

If an information is missing, you just leave it as an empty string.


In [ ]:
# Inizialize dataframe
columns = ['courseName', 'universityName','facultyName','isItFullTime','description','startDate','fees','modality','duration','city','country','adminsitration','url']
data = []

In [ ]:
# For all the folders
for number in tqdm(range(1,401), desc= "Creating tsv files", unit="URL"):

    # Path of folder
    folder_path = f'/Users/Francesco/Desktop/ADM/HW3/Page{number}'

    # List of files in directory
    file_list = os.listdir(folder_path)
    file_list = [f for f in file_list if f.endswith('.html')] # On Mac there is a file called .DS_store, we don't want it

    # For each file
    for i,file_name in enumerate(file_list):
        
        # Full path of file (folder path + file path)
        file_path = os.path.join(folder_path, file_name)

        # Read file
        with open(file_path, 'rb') as file:
            file_content = file.read()

        # Get infos of the page into a list
        content = crawler.extract_MSc_content(file_content)

        # Create tsv file using name of the course
        file_name = f'course_{i}'
        file_path = os.path.join(folder_path, file_name)
        
        with open(f'{file_path}.tsv','w') as tsvfile:
            tsvfile.write(f'{content[0]} \t {content[1]} \t {content[2]} \t {content[3]} \t {content[4]} \t {content[5]} \t {content[6]} \t {content[7]} \t {content[8]} \t {content[9]} \t {content[10]} \t {content[11]} \t {content[12]}')

In [ ]:
# Merging the .tsv files into one df
for number in tqdm(range(1,401), desc= "Merging tsv files", unit="URL"):

    # Path of folder
    folder_path = f'/Users/Francesco/Desktop/ADM/HW3/Page{number}'

    # List of files in directory
    file_list = os.listdir(folder_path)
    file_list = [f for f in file_list if f.endswith('.tsv')] # On Mac there is a file called .DS_store, we don't want it

    # For each file
    for file_name in file_list:

        # Full path of file
        file_path = os.path.join(folder_path, file_name)

        # Read the .tsv file
        df = pd.read_csv(file_path, sep='\t',names=columns)

        # Append df into a list
        data.append(df)

# Concat all the df into one       
merged_df = pd.concat(data, ignore_index=True)

# Download df
merged_df.to_csv('/Users/Francesco/Desktop/ADM/HW3/df.csv', index=False)

## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the courses that match the query.

In [18]:
import pandas as pd
df = pd.read_csv('/home/theballer/Desktop/Sapienza Courses/ADM/ADM-HW3/df.csv')

### 2.0 Preprocessing 

### 2.0.0)  Preprocessing the text

First, you must pre-process all the information collected for each MSc by:

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed
   
For this purpose, you can use the [`nltk library](https://www.nltk.org/).

In [19]:
from nltk.stem import *
import nltk
from nltk.corpus import stopwords
#create the instance of the stemmer
stemmer = PorterStemmer()
#download the nltk stopwords
nltk.download('stopwords')
#make sure that they are english words
lst_stopwords = stopwords.words('english')
#drop potential null values from the description column
df = df.dropna(subset=['description'])
#uses apply method with list comprehension to tokenize each row and stem each word as well as filter them on alphanumeric and prevent stopwords
df['descr_clean'] = df.description.apply(lambda row: [stemmer.stem(word) for word in nltk.word_tokenize(row) if not word in lst_stopwords and word.isalnum()])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/theballer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import pandas as pd
import re

df['fees'] = df['fees'].apply(lambda row: str(row).replace(',', ''))

def extract_largest_amount(text):
    # Define the regular expression pattern
    # This pattern looks for amounts with or without a space before/after the currency symbol
    # It only looks for dollars, pounds, and euros
    currency_pattern = r'(\$\s?\d+(?:\.\d+)?|\d+(?:\.\d+)?\s?€|\d+(?:\.\d+)?\s?£|£\s?\d+(?:\.\d+)?|\d+(?:\.\d+)?\s?\$|\d+(?:\.\d+)?\s?€|\d+(?:\.\d+)?\s?£|\£\s?\d+(?:\.\d+)?|\€\s?\d+(?:\.\d+)?|\d+(?:\.\d+)?\s?\€)'

    # Find all matches
    matches = re.findall(currency_pattern, text)

    # Convert matches to numeric values for comparison
    amounts = []
    for match in matches:
        # Remove currency symbols and non-numeric characters, convert to float
        amount = float(re.sub(r'[^\d.]', '', match))
        amounts.append((amount, match))

    # Find the largest amount
    if amounts:
        return max(amounts)[1]
    else:
        return None

# Apply the function to the DataFrame
df['cleaned_fees'] = df['fees'].apply(extract_largest_amount)

In [21]:
#the number of numerical values that did not catch out regex
num_values_cleaned_fees = df['cleaned_fees'].notna().sum()
num_values_fees = len(df[df['fees'].astype(str).str.contains(r'\d')])
print(f"There are {num_values_fees} of rows that have some digits, my algorithm found {num_values_cleaned_fees} rows after cleaning fees")

There are 1627 of rows that have some digits, my algorithm found 1217 rows after cleaning fees


In [24]:
not_caught_values = df[df['fees'].astype(str).str.contains(r'\d') & df['cleaned_fees'].isna()]
not_caught_values.head()
#they are other currencies like ISK, that is quite difficult to catch

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,adminsitration,url,descr_clean,cleaned_fees
58,Coastal and Marine Management,University Centre of the West Fjords,Masters Program,Full time & Part time,Coastal and Marine Management http://www.uw.i...,September,The annual fee is ISK 150.000 for students fr...,MSc,18 months,Isafjorour,Iceland,,https://www.findamasters.com/masters-degrees/...,"[coastal, marin, manag, http, intern, interdis...",None
87,Doctor of Public Health,Harvard University,Harvard T.H. Chan School of Public Health,Full time,﻿Learn to translate knowledge into powerful r...,Any Month,All students admitted to the DrPH program inc...,"MSc, Prof Doctorate, Other",3 years full time,Boston,USA,On Campus,https://www.findamasters.com/masters-degrees/...,"[translat, knowledg, power, result, leader, pu...",None
98,Engineering Business Management online MSc,University of Bath,University of Bath Online,Part time,Develop an in-depth understanding of engineer...,"September, January",The Engineering Business Management online MS...,MSc,2 years and 3 months full time,Bath,United Kingdom,Online,https://www.findamasters.com/masters-degrees/...,"[develop, understand, engin, technolog, leader...",None
185,Master in Global Logistics and Supply Chain Ma...,Kuehne Logistics University - KLU,Business and Management,Full time & Part time,Our MSc in Global Logistics and Supply Chain ...,See Course,EUR 7390.00 per semester (Standard and Part-t...,MSc,2 Years,Hamburg,Germany,On Campus,https://www.findamasters.com/masters-degrees/...,"[our, msc, global, logist, suppli, chain, mana...",None
346,MSc International Management,University of Nottingham Ningbo China,International Masters Courses,Full time,The MSc International Management programme fu...,September,Domestic course fees: 100000 RMB / year Inter...,MSc,12 months full time,Ningbo,China,On Campus,https://www.findamasters.com/masters-degrees/...,"[the, msc, intern, manag, programm, furnish, s...",None


### 2.0.1) Preprocessing the fees column

Moreover, we want the field ```fees``` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a ```float``` column renamed ```fees (CHOSEN COMMON CURRENCY)```.


In [25]:
import pandas as pd
from forex_python.converter import CurrencyRates
#Let's calculate the rate of each currencies compared to EUR to save time in computaton
c = CurrencyRates()
dollar = c.convert('USD', 'EUR', float(1))
pound = c.convert('GBP', 'EUR', float(1))
# Define a function to convert currencies to euros using an API
def convert_to_euros(row):
    if pd.notnull(row['cleaned_fees']):
        c = CurrencyRates()

        # Ensure the "largest_amount" column is treated as a Series
                
        cleaned_fees_series = pd.Series(row['cleaned_fees'])

        # Replace specific currency symbols with an empty string
        cleaned_fees_series = cleaned_fees_series.replace({'€': '', '$': '', '£': ''}, regex=True)

        # Extract the amount from the modified "cleaned_fees" column
        amount = cleaned_fees_series.str.extract(r'(?P<amount>\d+(\.\d+)?)')['amount'].values[0]

        # Check for the presence of specific currency symbols
        if '€' in row['cleaned_fees']:
            return float(amount)  # No conversion needed for euros
        elif '$' in row['cleaned_fees']:
            return round(float(amount) * dollar, 2)
        elif '£' in row['cleaned_fees']:
            return round(float(amount) * pound, 2)
        else:
            return None  # Handle unknown currencies

    else:
        return None

# Apply the conversion function to the DataFrame
df['fees (EUR)'] = df.apply(convert_to_euros, axis=1)

In [26]:
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,adminsitration,url,descr_clean,cleaned_fees,fees (EUR)
0,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,UK: £18000 (Total) International: £34750 (Tot...,MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[busi, govern, reli, sound, financi, knowledg,...",£34750,39871.49
1,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuro...",Full time & Part time,Join us for an online session for prospective...,September,Please see the university website for further...,MSc,One year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[join, us, onlin, session, prospect, student, ...",None,NaN
2,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[embark, profession, account, career, academ, ...",None,NaN
3,"Agricultural, Environmental and Resource Econo...",University of Helsinki,International Masters Degree Programmes,Full time,Goal of the pro­grammeAre you looking forward...,September,Tuition fee per year (non-EU/EEA students): 1...,MSc,2 years,Helsinki,Finland,On Campus,https://www.findamasters.com/masters-degrees/...,"[goal, look, forward, futur, expert, agricultu...",15000 €,15000.00
4,Applied Economics (Banking and Financial Marke...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and rec...,"September, January",Cost per 10 credits £722* (10% alumni discoun...,MSc,2 years and 6 months full time,Bath,United Kingdom,Online,https://www.findamasters.com/masters-degrees/...,"[from, polit, uncertainti, financ, recruit, de...",£722,828.41


### 2.1. Conjunctive query
For the first version of the search engine, we narrowed our interest to the __description__ of each course. It means that you will evaluate queries only concerning the course's description.


In [27]:
vocabulary = set()
df.descr_clean.apply(lambda row: [vocabulary.add(word) for word in row]) 
#creating a set of all words in the corpus

0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5997    [None, None, None, None, None, None, None, Non...
5998    [None, None, None, None, None, None, None, Non...
5999    [None, None, None, None, None, None, None, Non...
6000    [None, None, None, None, None, None, None, Non...
6001    [None, None, None, None, None, None, None, Non...
Name: descr_clean, Length: 6001, dtype: object

In [28]:
#creating a dataframe with all words in the vocabulary
terms = pd.DataFrame(data=list(vocabulary), columns=['term'])

In [30]:
from tqdm.notebook import tqdm
#tqdm is used to check the progress
tqdm.pandas()
#creating column inverse that will contain the term with the list of idecies of the documents in which this term is contained
terms['reverse'] = terms.term.progress_apply(lambda item: list(df.loc[df.descr_clean.apply(lambda row: item in row)].index))

  0%|          | 0/8853 [00:00<?, ?it/s]

In [31]:
#saving this file for further use
terms.to_json('/home/theballer/Desktop/Sapienza Courses/ADM/ADM-HW3/terms.json')

### 2.1.1) Create your index!

Before building the index, 
* Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
where _document\_i_ is the *id* of a document that contains that specific word.

__Hint:__ Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

In [32]:
import pandas as pd
terms = pd.read_json('/home/theballer/Desktop/Sapienza Courses/ADM/ADM-HW3/terms.json')

#### 2.1.2) Execute the query
Given a query input by the user, for example:

```
advanced knowledge
```

The Search Engine is supposed to return a list of documents.

##### What documents do we want?
Since we are dealing with conjunctive queries (AND), each returned document should contain all the words in the query.
The final output of the query must return, if present, the following information for each of the selected documents:

* `courseName`
* `universityName`
* `description`
* `URL`

__Example Output__ for ```advanced knowledge```: (please note that our examples are made on a small batch of the full dataset)

<p align="center">
<img src="img/output1.png" width = 1000>
</p>

If everything works well in this step, you can go to the next point and make your Search Engine more complex and better at answering queries.


In [33]:
from search_engine_1 import search_engine_1
query = 'advanced knowledge'
search_engine_1_df = search_engine_1(df, terms, query, stemmer, lst_stopwords)

In [34]:
display(search_engine_1_df[['courseName', 'universityName', 'description', 'url']])

,courseName,universityName,description,url
0,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fina...,https://www.findamasters.com/masters-degrees/...
1,Addictions MSc,King’s College London,Join us for an online session for prospective...,https://www.findamasters.com/masters-degrees/...
7,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stu...,https://www.findamasters.com/masters-degrees/...
51,Civil Engineering MSc,University of Greenwich,Meet the future demands of the construction i...,https://www.findamasters.com/masters-degrees/...
77,Economics - MSc,University of Leeds,Our MSc Economics allows you to apply economi...,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...
5934,Master Sociology – Social and Economic Psychol...,University of Cologne,This programme provides you with:a solid foun...,https://www.findamasters.com/masters-degrees/...
5937,Master's course in Cognitive Science,University of Trento,Course overviewThe two-year CIMeC Master’s in...,https://www.findamasters.com/masters-degrees/...
5946,Masters in Economics,University of Lisbon,OBJECTIVESThe MSc in Economics aims to provid...,https://www.findamasters.com/masters-degrees/...
5953,Master's in Global and European Politics,European School of Political and Social Scien...,Europe and the EU in a changing worldOur inte...,https://www.findamasters.com/masters-degrees/...


### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the *top-k* (the choice of *k* it's up to you!) documents related to the query. In particular:

* Find all the documents that contain all the words in the query.
* Sort them by their similarity with the query.
* Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than _k_. You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

To solve this task, you must use the *tfIdf* score and the _Cosine similarity_. The field to consider is still the `description`. Let's see how.



#### 2.2.1) Inverted index
Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word, you want the list of documents in which it is contained and the relative *tfIdf* score.

__Tip__: *TfIdf* values are invariant for the query. Due to this reason, you can precalculate and store them accordingly.


#### 2.2.2) Execute the query

In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the *tfIdf* representations of the documents.

Given a query input by the user, for example:
```
advanced knowledge
```
The search engine is supposed to return a list of documents, __ranked__ by their Cosine Similarity to the query entered in the input.

More precisely, the output must contain:
* `courseName`
* `universityName`
* `description`
* `URL`
* The similarity score of the documents with respect to the query (float value between 0 and 1)
  
__Example Output__ for ```advanced knowledge```:

<p align="center">
<img src="img/output2.png" width = 1000>
</p>


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
#create instance of tfidfVectorizer
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(df.descr_clean)
#from sparse(omitting the values that contain zeros) to dense matrix
result_dense = results.todense()
#creating the dataframe for better understanding and futher use
tfidf_data = pd.DataFrame(result_dense.tolist(), index=df.index, columns=tfidf.get_feature_names_out())

/home/theballer/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
#pd.set_option('display.max_columns', 7000)
#saving the dataframe so we do not need to calculate the inverse dictionary again
tfidf_data.to_csv('/home/theballer/Desktop/Sapienza Courses/ADM/ADM-HW3/tfidf_data.csv', index=False)

In [38]:
from search_engine_2 import search_engine_2
query = 'advanced knowledge'
result = search_engine_2(df, terms, lst_stopwords, query, K=10)
result

/home/theballer/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,courseName,universityName,description,url,Similarity
0,Advancing Practice (PgCert/PgDip/MSc),Robert Gordon University,The MSc Advancing Practice course will equip ...,https://www.findamasters.com/masters-degrees/...,0.996266
1,Advanced Structural Engineering MSc (Campus),Aberdeen University,This advanced programme allows you to explore...,https://www.findamasters.com/masters-degrees/...,0.996266
2,"Advanced Clinical Practice MSc, PGDip, PGCert",King’s College London,Advanced Clinical Practice is for experienced...,https://www.findamasters.com/masters-degrees/...,0.996266
3,International Tourism Management MSc,Swansea University,Are you interested in a career in tourism whe...,https://www.findamasters.com/masters-degrees/...,0.993414
4,"Information Systems, Master Programme (60 cred...",Linnaeus University,Information systems offer opportunities for e...,https://www.findamasters.com/masters-degrees/...,0.993414
5,"Health, Wellbeing and Sustainable Buildings MSc",University College London,Register your interest in graduate study at U...,https://www.findamasters.com/masters-degrees/...,0.993414
6,Health and Management MSc (PGCert PGDip),University of Brighton,The Health and Management MSc gives you the o...,https://www.findamasters.com/masters-degrees/...,0.993414
7,Financial Mathematics - MSc,University of Leeds,This degree provides you with knowledge of ad...,https://www.findamasters.com/masters-degrees/...,0.993414
8,Finance and Investment - MSc,University of Leeds,Global capital and money markets are rapidly ...,https://www.findamasters.com/masters-degrees/...,0.993414
9,Finance and Data Analytics (MSc),"Birkbeck, University of London",The financial world’s demand for innovation a...,https://www.findamasters.com/masters-degrees/...,0.993414


## 3. Define a new score!

Now it's your turn: build a new metric to rank MSc degrees.

Practically:

1. The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
2. Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the ```description``` field of the documents; you __can__ use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

__N.B.:__ You have to define a __scoring function__, not a filter! 

The output, must contain:

* `courseName`
* `universityName`
* `description`
* `URL`
* The  __new__ similarity score of the documents with respect to the query

Are the results you obtain better than with the previous scoring function? **Explain and compare results**.


## 4. Visualizing the most relevant MSc degrees

Using maps can help people understand how far one university is from another so they can plan their academic careers more adequately. Here, we challenge you to show a map of the courses found with the score defined in point 3. You should be able to identify at least the *city* and *country* for each MSc degree. You can find some ideas on how to create maps in Python [here](https://plotly.com/python/maps/) and [here](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621) but you will maybe need further information for a proper visualization, like coordinates (latitude and longitude). You can retrieve this data using various tools:

1. [Here](https://medium.com/@manilwagle/geocoding-the-world-using-google-api-and-python-1f6b6fb6ca48) you can find a helpful tutorial on how to encode geo-informations using Google API in Python (this tool can also be used in [Google Sheets](https://handsondataviz.org/geocode.html))
2. You can collect a list of unique places in the format (City, Country) and ask chatGPT (or, as usual, any other LLM chatbot) to provide you with a list of corresponding representative coordinates
3. Explore and find the best solution for your case!
   
Once you defined your visualization strategy, include a way to encode fees in your charts. The map should show (with a proper legend) different courses and associated taxation: the user wants a glimpse not only of how far he will need to move but also of how much it will cost him!


## 5. BONUS: More complex search engine 

__IMPORTANT:__ This is a bonus step, so it's <ins>not mandatory</ins>. You can get the maximum score also without doing this. We will take this into account, **only if** the rest of the homework has been completed.

For the Bonus part, we want to ask you more sophisticated search engine. Here we want to let users issue more complex queries. The options of this new search engine are: 
1. Give the possibility to specify queries for the following features (the user should have the option to issue __none or all of them__): 
* `courseName`
* `universityName`
* `universityCity`
2. Specify a range for the __fees__ to retrieve only MSc whose taxation is in that range.
3. Specify a list of __countries__ which the search engine should only return the courses taking place in city within those countries.
4. Filter based on the courses that have already started. 
5. Filter based on the presence of online modality. 

__Note 1__: You should be aware that you should give the user the possibility <ins>to select any</ins> of the abovementioned options. How should the user use the options? We will accept __any manual__ that you provide to the user. 

__Note 2__: As you may have realized from __1st option__, you need to build <ins>inverted indexes</ins> for those values and return all of the documents that have the similarity <ins>more than 0</ins> concerning the given queries. Choose a __logical__ way to aggregate the similarity coming from each of them and explain your idea in detail.

__Note 3__: The options <ins>other than 1st</ins> one can be considered as __filtering criteria__ so the retrieved documents <ins>must respect all</ins> of those filters. 

The output must contain the following information about the places:

* `courseName`
* `universityName`
* `url`

## 6. Command Line Question

As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

Note: To answer the question in this section, you must strictly use command line tools. We will reject any other method of response. The final script must be placed in CommandLine.sh.

First, take the course_i.tsv files you created in point 1 and merge them using Linux commands (Hint: make sure that the first row containing the column names appears only once).

Now that you have your merged file named merged_courses.tsv, use Linux commands to answer the following questions:
- Which country offers the most Master's Degrees? Which city?
- How many colleges offer Part-Time education?
- Print the percentage of courses in Engineering (the word "Engineer" is contained in the course's name).

__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.



## 7. Algorithmic Question 
Leonardo is an intern at a company. He is paid based on the total number of hours he has worked. They agreed __d__ days ago that Leonardo could not work less than $minTime_i$ or more than $maxTime_i$ hours per <ins>i-th</ins> day. Furthermore, he was warned by HR that on his last day at the company, he should provide a detailed report on how many hours he worked <ins>each day</ins> for the previous d days.

Today is the day Leonardo should report to HR, but the problem is that he <ins>didn't</ins> account for how many hours he put in for each day, so he only has the __total sum of the hours__ ($sumHours$) he put in total in these d days. He believes that if he creates a report in which each number $dayHours_i$ corresponds to the __total hours he worked on the i-th day__ while satisfying the HR limitations and the total sum of all $dayHours_i$ equals $sumHours$, he would be fine.

He cannot create such a report independently and requests your assistance. He will give you the number of days $d$, total hours spent $sumHours$, and the HR limitations for each day $i$, and he wants you to assist him in determining whether it is possible to create such a fake report. If that is possible, make such a report. 

**Input**

The first line of input contains two integers __d__, $sumHours$ - the number of days Leonardo worked there and the total number of hours he worked for the company. Each of the following __d__ lines contains two integer numbers $minTime_i$ and $maxTime_i$ - the minimum and maximum hours he can work on the $i_{th}$ day. 

**Output**

If such a report cannot be generated, print 'NO' in one output line. If such a report is possible, print 'YES' in the output and d numbers - the number of hours Leonardo spent each day - in the second line. If more than one solution exists, print any of them. 

__Input 1__
```
2 5
0 1
3 5
```
__Output 1__
```
YES
1 4 
```
---
__Input 2__
```
1 1
5 6
```
__Output 2__
```
NO
```

1. Implement a code to solve the above mentioned problem. 

In [ ]:
def report():
    # 'value'    --> is a list that contains the total number of days worked and the total number of hours worked 
    # 'remaning' --> is a list used to store the condition 'YES' or 'NO' and 
    #                the remaining hours of work that are not assign to a specific day
    # 'i'        --> is an index
    value, remaning, i = list(map(int, input().split())), ["", 0], 0
    
    # We create a list of lists composed of a set of values containing the minimum hours worked, 
    # the maximum hours worked and the hours actually worked initialized to the minimum value
    # Every item in this list of list is a different day
    days = [[minimo, massimo, minimo] for _ in range(value[0]) for minimo, massimo in [map(int, input().split())]]
    
    # Check if the sum of maximum possible working hours for all days is greater or equal to 
    # the total number of hours worked and if the sum of minumum working hours is lower than 
    # the total of hours worked
    # --> if the condition is TRUE than we can generate a report with the desired features 
    #     --> we set the values of 'remaning' equal to 'YES' and equal to the the remaining hours of work 
    #         that are not assigned to a specific day
    # --> if the condition is FALSE than we can not generate a report with the desired features
    #     --> we set the values of 'remaning' equal to 'NO' and 0 
    remaning = ["YES", (value[1] - sum(item[0] for item in days))] if (sum(item[1] for item in days) >= value[1]) & (sum(item[1] for item in days) <= value[1]) else ["NO", 0] 
    
    # If we can not generate a report with the desired features then we return 'NO'
    if remaning[0] == "NO": return remaning[0]
    
    # If we get to this point it is implied that we can generate a report with the desired features
    # As long as the number of hours that are not assigned is greater than 0 it means that in some days 
    #    we have worked more than the minimum number of hours
    while remaning[1] > 0:
        # Update the working hours for the current day to the maximum
        # We reduce the number of hours that we have not yet assigned 
        #    by the number of hours we have just added to the i-th day (maximum-minimum)
        # We add 1 to the index
        days[i][2], remaning[1], i = days[i][1], remaning[1] - (days[i][1] - days[i][0]), i + 1
    
    # If there is a negative remaining of hours that we have not yet assigned 
    #    it means that we added too many hours to the last day set to the maximum hours 
    # So we remove this excess from the last day
    # --> we use the + operator because the remaining number is negative
    if remaning[1] < 0:
        days[i - 1][2] = days[i - 1][2] + remaning[1]
    
    # Return the result in the specified format
    return f'{remaning[0]}\n{" ".join(map(str, list(zip(*days))[2]))}'

# Print the result of the report-function
print(report())

2. What is the __time complexity__ (the Big O notation) of your solution? Please provide a <ins>detailed explanation</ins> of how you calculated the time complexity.

3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the <ins>two differ</ins>, which one is right? (why?)
   
4. What do you think of the __optimality__ of your code? Do you believe it is optimal? Can you improve? Please <ins>elaborate</ins> on your response. 